# An inferential problem: The Gender Wage Gap

## Data analysis

In [ ]:
import pandas as pd
import numpy as np
import pyreadr as rr # package to use data form R format
import math

In [ ]:
#!pip install pyreadr==0.4.2

In [ ]:
#rdata_read = pyreadr.read_r("../../data/wage2015_subsample_inference.Rdata")

data  = pd.read_csv(r'C:/Users/Frank/Downloads/wage2015_subsample_inference.csv')

# Extracting the data frame from rdata_read
#data = rdata_read[ 'data' ]
data['occ']=pd.Categorical(data.occ)
data['occ2']=pd.Categorical(data.occ2)
data['ind']=pd.Categorical(data.ind)
data['ind2']=pd.Categorical(data.ind2)

data

In [ ]:
data.shape

In [ ]:
data.info()

***Variable description***

- occ : occupational classification
- ind : industry classification
- lwage : log hourly wage
- sex : gender (1 female) (0 male)
- shs : some high school
- hsg : High school graduated
- scl : Some College
- clg: College Graduate
- ad: Advanced Degree
- ne: Northeast
- mw: Midwest
- so: South
- we: West
- exp1: experience

In this case, we Focus on the subset of college-advanced-educated workers. he analysis should be analogous to what we’ve presented – explaining basic, control and partialling out model, generating point estimates and standard errors.

In [ ]:
data = data[(data.scl == 1) | (data.ad == 1) | (data.clg == 1)]
data

In [ ]:
data.shape

To start our (causal) analysis, we compare the sample means given gender:

In [ ]:
Z = data[ ["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] ]

data_female = data[data[ 'sex' ] == 1 ]
Z_female = data_female[ ["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] ]

data_male = data[ data[ 'sex' ] == 0 ]
Z_male = data_male[ [ "lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1" ] ]


table = np.zeros( (12, 3) )
table[:, 0] = Z.mean().values
table[:, 1] = Z_male.mean().values
table[:, 2] = Z_female.mean().values
table_pandas = pd.DataFrame( table, columns = [ 'All', 'Men', 'Women']) # from table to dataframe
table_pandas.index = ["Log Wage","Sex","Some High School","High School Graduate","Some College","Gollage Graduate","Advanced Degree", "Northeast","Midwest","South","West","Experience"]
table_html = table_pandas.to_html() # html format

table_pandas

In [ ]:
print( table_html )

In particular, the table above shows that the difference in average logwage between men and women is equal to  0,075

In [ ]:
data_female['lwage'].mean()- data_male['lwage'].mean()

Thus, the unconditional gender wage gap is about 7.5% for the group of never married workers (women get paid less on average in our sample). We also observe that never married working women are relatively more educated than working men and have lower working experience.

In [ ]:
rdata_read = rr.read_r(r"../../../data/wage2015_subsample_inference.Rdata")

xx
# Extracting the data frame from rdata_read
data = rdata_read[ 'data' ]


data.shape

This unconditional (predictive) effect of gender equals the coefficient $\beta$ in the univariate ols regression of $Y$ on $D$:

\begin{align}
\log(Y) &=\beta D + \epsilon.
\end{align}

We verify this by running an ols regression in R.

In [ ]:
pip install statsmodels

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
nocontrol_model = smf.ols( formula = 'lwage ~ sex', data = data )
nocontrol_est = nocontrol_model.fit().summary2().tables[1]['Coef.']['sex']
nocontrol_est
nocontrol_se2 = nocontrol_model.fit().summary2().tables[1]['Std.Err.']['sex']


# robust standar erros
HCV_coefs = nocontrol_model.fit().cov_HC0
nocontrol_se = np.power( HCV_coefs.diagonal() , 0.5)[1]
nocontrol_se

# print unconditional effect of gender and the corresponding standard error

print( f'The estimated gender coefficient is {nocontrol_est} and the corresponding standard error is {nocontrol_se2}' )
print( f'The estimated gender coefficient is {nocontrol_est} and the corresponding robust standard error is {nocontrol_se}','\n' )


# confidence interval
nocontrol_model.fit().conf_int( alpha=0.05 ).loc[['sex']]


Note that the standard error is computed with the *R* package *sandwich* to be robust to heteroskedasticity. 


Next, we run an ols regression of $Y$ on $(D,W)$ to control for the effect of covariates summarized in $W$:

\begin{align}
\log(Y) &=\beta_1 D  + \beta_2' W + \epsilon.
\end{align}

Here, we are considering the flexible model from the previous lab. Hence, $W$ controls for experience, education, region, and occupation and industry indicators plus transformations and two-way interactions.

Let us run the ols regression with controls.

## Ols regression with controls

In [ ]:
flex = 'lwage ~ sex + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)'

# The smf api replicates R script when it transform data
control_model = smf.ols( formula = flex, data = data )
control_est = control_model.fit().summary2().tables[1]['Coef.']['sex']

print(control_model.fit().summary2().tables[1])

HCV_coefs = control_model.fit().cov_HC0
control_se = np.power( HCV_coefs.diagonal() , 0.5)[42]  # error standard for sex's coefficients 

control_se

print( f"Coefficient for OLS with controls {control_est} and the corresponding robust standard error is {control_se}" )

# confidence interval
control_model.fit().conf_int( alpha=0.05 ).loc[['sex']]


In [ ]:
control_model 

The estimated regression coefficient  𝛽1≈−0.0676 measures how our linear prediction of wage changes if we set the gender variable  𝐷 from 0 to 1, holding the controls  𝑊 fixed. We can call this the predictive effect (PE), as it measures the impact of a variable on the prediction we make. Overall, we see that the unconditional wage gap of size  8 % for women decreases to about  7
 % after controlling for worker characteristics.


Next, we are using the Frisch-Waugh-Lovell theorem from the lecture partialling-out the linear effect of the controls via ols.

## Partialling-Out using ols

In [ ]:
# models
# model for Y
flex_y = 'lwage ~  (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)'
# model for D
flex_d = 'sex ~ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)' 

# partialling-out the linear effect of W from Y
t_Y = smf.ols( formula = flex_y , data = data ).fit().resid

# partialling-out the linear effect of W from D
t_D = smf.ols( formula = flex_d , data = data ).fit().resid


data_res = pd.DataFrame( np.vstack(( t_Y.values , t_D.values )).T , columns = [ 't_Y', 't_D' ] )


# regression of Y on D after partialling-out the effect of W
partial_fit =  smf.ols( formula = 't_Y ~ t_D' , data = data_res ).fit()
partial_est = partial_fit.summary2().tables[1]['Coef.']['t_D']


# standard error
HCV_coefs = partial_fit.cov_HC0
partial_se = np.power( HCV_coefs.diagonal() , 0.5)[1]

print( f"Coefficient for D via partialling-out {partial_est} and the corresponding robust standard error is {partial_se}" )

# confidence interval
partial_fit.conf_int( alpha=0.05 ).loc[['t_D']]


In [ ]:
#np.vstack(( t_Y.values , t_D.values )).T

data_res = pd.DataFrame( np.vstack(( t_Y.values , t_D.values )).T , columns = [ 't_Y', 't_D' ] )
data_res

Again, the estimated coefficient measures the linear predictive effect (PE) of $D$ on $Y$ after taking out the linear effect of $W$ on both of these variables. This coefficient equals the estimated coefficient from the ols regression with controls.

We know that the partialling-out approach works well when the dimension of $W$ is low
in relation to the sample size $n$. When the dimension of $W$ is relatively high, we need to use variable selection
or penalization for regularization purposes. 


Use appropiate plots (i.e hist, barplots, scatter plots , pie plots, etc) to describe main varaibles (wage, log-wage, sex, some college, college graduate, avdanced degree, Experience)

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

box = sns.boxplot(x="sex", y="lwage", data=data ,palette='pastel')
plt.xlabel('Sexo')
plt.ylabel('Logaritmo del salario por hora')

# The real wage quartiles are increasing with the educational level.
# Lower salary dispersion for the postgraduate level. pastel

El promedio del logaritmo del salario es mayor para los hombres que para las mujeres

In [ ]:
data['wage'].plot(kind = 'hist', bins = 50, figsize = (8,6))
plt.title('Salario por hora')

txt="Elaboración propia"  
plt.figtext(0.5, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=12)
plt.xlim(0, 150)
plt.show()

El salario por hora mas frecuente es el de 20 y hay una distribucion con cola a la izquierda. El salario mas alto es el de 130.

Para las mujeres, 

In [ ]:
sns.regplot(data=data, x="exp1", y="lwage", x_bins=np.arange(0, 40, 5), order=1)

In [ ]:
#A mas anos de experiencia, las personas consiguen un mayor salario.

In [ ]:
sns.set_style("white")
gridobj = sns.lmplot(x="exp1", y="lwage", 
                     data=data, 
                     height=7, 
                     robust=True, 
                     palette='Set1', 
                     col="sex",
                     scatter_kws=dict(s=60, linewidths=0.7, edgecolors='black'))

gridobj.set(xlim=(0, 80), ylim=(0, 10))
plt.show()

A simple vista, se ve que la experiencia impacta positivamente en el salario. Pero, en las mujeres ese efecto es menor que en los hombres debido a las brechas de genero persistentes en el mercado laboral.

Plot the confidence Interval of sex's coefficient for a different models (basic, control, and partially out). All three coefficients must be in one figure. Explain what you find.

In [ ]:
coef_df = pd.DataFrame({
    'coef': [nocontrol_est, control_est, partial_est],
    'err': [nocontrol_se2, control_se, partial_se],
    'varname': ['No control', 'Con controles', 'Parcialmente fuera']
})

# Reorganizar las columnas
coef_df = coef_df[['coef', 'err', 'varname']]

# Mostrar el dataframe
print(coef_df)

fig, ax = plt.subplots(figsize=(8, 3))
coef_df.plot(x='varname', y='coef', kind='bar', 
             ax=ax, color='none', 
             yerr='err', legend=False)
ax.set_ylabel('')
ax.set_xlabel('')
ax.scatter(x=pd.np.arange(coef_df.shape[0]), 
           marker='s', s=120, 
           y=coef_df['coef'], color='black')
ax.axhline(y=0, linestyle='--', color='black', linewidth=4)
ax.xaxis.set_ticks_position('none')
_ = ax.set_xticklabels(['No control', 'Con controles', 'Parcialmente fuera'], 
                       rotation=0, fontsize=16)


Los coeficientes son mayores para los modelos de Con controles y parcialmente fuera, cuando se restringe la muestra para las personas con educacion avanzada. Asimismo, el error se minimiza cuandos ewe usan los dos ultimos mnodelos, por lo que el intervalo de confianza es menor.

You will also include a replication of the next figure for both groups, female and male.You will have only two plotted lines (Actual/Predicted(fitted)) for these College-educated workers. You have to create two separate figures, one for female and one for male. Could you explain the different patterns that you find?

In [ ]:
# In[ ]:





# Los patrones que se encuentran son que los salarios son mayores para las personas que asisten a la universidad que de los que asisten solo a la secundaria. Luego, la relacion de los anos de experiencia en el salario solo es positiva, tanto para universidad y secundaria, hasta antes de que se llegue a los 35 anos de experiencia.


data_hsg = data[data['hsg'] == 1]
data_clg = data[data['clg'] == 1]
data_scl = data[data['scl'] == 1]

data_clgm = data_clg[data_clg['sex'] == 0]  # Hombres
data_clgf = data_clg[data_clg['sex'] == 1]  # Mujeres



import pandas as pd

# Tabla_hsg
Tabla_hsg = data_hsg.groupby('exp2').agg(Promlwageo=('lwage', 'mean')).reset_index()

nivel_hsg = sorted(data_hsg['exp2'].unique())

Promedio = []
for nivel in nivel_hsg:
    Promedio.append(data_hsg[data_hsg['exp2'] <= nivel]['lwage'].mean())

Tabla_hsg['PromMov'] = Promedio
print(Tabla_hsg.head())

# Tabla_clg
Tabla_clg = data_clg.groupby('exp2').agg(Promlwageo=('lwage', 'mean')).reset_index()

Tabla_clgm = data_clg.groupby('exp2').agg(Promlwageo=('lwage', 'mean')).reset_index()
Tabla_clgf = data_clg.groupby('exp2').agg(Promlwageo=('lwage', 'mean')).reset_index()

nivel_clg = sorted(data_clg['exp2'].unique())
nivel_clgm = sorted(data_clgm['exp2'].unique())
nivel_clgf = sorted(data_clgf['exp2'].unique())

Promedio = []
for nivel in nivel_clg:
    Promedio.append(data_clg[data_clg['exp2'] <= nivel]['lwage'].mean())

Tabla_clg['PromMov'] = Promedio
Tabla_clgm['PromMov'] = Promedio
Tabla_clgf['PromMov'] = Promedio
print(Tabla_clg.head())

import statsmodels.api as sm
import pandas as pd

# Definir la fórmula del modelo
formula = 'lwage ~ sex + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)'

# Ajustar el modelo
control_fit1 = sm.formula.ols(formula, data=data).fit()

# Hacer predicciones
predict = control_fit1.predict(data)

# Añadir las predicciones al DataFrame original
data['Predict'] = predict

# Filtrar datos para scl y clg
data_sclP = data[data['scl'] == 1]
data_clgP = data[data['clg'] == 1]
data_hsgP = data[data['hsg'] == 1]

data_clgPm = data_clgP[data_clgP['sex'] == 0]  # Hombres
data_clgPf = data_clgP[data_clgP['sex'] == 1]  # Mujeres

import pandas as pd
###########################################################
# Using "sclP"
Tabla_hsgP = data_hsgP.groupby('exp2')['Predict'].mean().reset_index()

nivel_hsgP = sorted(data_hsgP['exp2'].unique())

Promedio = []
for nivel in nivel_hsgP:
    Promedio.append(data_hsgP[data_hsgP['exp2'] <= nivel]['Predict'].mean())

Tabla_hsgP['PromMovP'] = Promedio
print(Tabla_hsgP.head())

# Repeat for "clgP"
Tabla_clgP = data_clgP.groupby('exp2')['Predict'].mean().reset_index()
Tabla_clgPf = data_clgPf.groupby('exp2')['Predict'].mean().reset_index()
Tabla_clgPm = data_clgPm.groupby('exp2')['Predict'].mean().reset_index()

nivel_clgP = sorted(data_clgP['exp2'].unique())

Promedio = []
for nivel in nivel_clgP:
    Promedio.append(data_clgP[data_clgP['exp2'] <= nivel]['Predict'].mean())

Tabla_clgP['PromMov'] = Promedio
Tabla_clgPf['PromMov'] = Promedio

Promediof = Promedio[:-1]
Tabla_clgPm['PromMov'] = Promediof
print(Tabla_clgP.head())

import matplotlib.pyplot as plt

# Datos
x = Tabla_clg['exp2']
x_3 = Tabla_clgPm['exp2']
y = Tabla_clg['PromMov']
y_3 = Tabla_clgPm['PromMov']

# Crear el gráfico
plt.plot(x, y, color='navy', linestyle='-', label="Actual CLG")

plt.plot(x_3, y_3, color='darkred', linestyle='--')

# Ajustes del gráfico
plt.ylim(3, 3.2)
plt.xlim(0, 15)
plt.xlabel("Years of Potential Experience")
plt.ylabel("Log Wage (or Wage Gap)")
plt.title("Figure 10: Comparison between actual and fitted for CLG and HSG Male")
plt.grid(linestyle='--', color='gray')

# Marcas de los ejes
plt.xticks(range(0, 16, 5))

# Leyenda
plt.legend(loc="upper right", fontsize=8)

# Mostrar el gráfico
plt.show()

# Datos
x = Tabla_clg['exp2']
x_3 = Tabla_clgPf['exp2']
y = Tabla_clg['PromMov']
y_3 = Tabla_clgPf['PromMov']

# Crear el gráfico
plt.plot(x, y, color='navy', linestyle='-', label="Actual CLG")

plt.plot(x_3, y_3, color='darkred', linestyle='--')

# Ajustes del gráfico
plt.ylim(3, 3.2)
plt.xlim(0, 15)
plt.xlabel("Years of Potential Experience")
plt.ylabel("Log Wage (or Wage Gap)")
plt.title("Figure 10: Comparison between actual and fitted for CLG and HSG Female")
plt.grid(linestyle='--', color='gray')

# Marcas de los ejes
plt.xticks(range(0, 16, 5))

# Leyenda
plt.legend(loc="upper right", fontsize=8)

# Mostrar el gráfico
plt.show()

Los patrones que se encuentran son que los salarios son mayores para las personas que asisten a la universidad que de los que asisten solo a la secundaria. Luego, la relacion de los anos de experiencia en el salario solo es positiva, tanto para universidad y secundaria, hasta antes de que se llegue a los 35 anos de experiencia. Luego, si diferenciamos entre sexo la diferencia es muy pequeña.

### Now, we will construct a prediction rule for (log) hourly wage **Y**, which depends linearly on job-relevant characteristics **X**:

$$
Y = \beta'X + \epsilon
$$

Our goals are:

- Predict wages using various characteristics of workers.
- Assess the predictive performance of a given model using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$.

Toward answering the latter, we measure the prediction quality of the two models via data splitting:

1. Randomly split the data into one training sample and one testing sample. Here we just use a simple method (stratified splitting is a more sophisticated version of splitting that we might consider).
2. Use the training sample to estimate the parameters of the Basic Model and the Flexible Model.

Before using the testing sample, we evaluate in-sample fit.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import random
import math

# Load the data
data = pd.read_csv("../../data/wage2015_subsample_inference.csv")
n = len(data)
# Define alpha values for Lasso
alphas = np.linspace(0.1, 0.5, 5)
print(alphas)
data.info()

In [ ]:
# Split data into training and test sets
np.random.seed(1)
random = np.random.randint(0,n, size=math.floor(n))
data["random"] = random
train = data[ : math.floor(n*4/5)]    # training sample
test =  data[ math.floor(n*4/5) : ]   # testing sample
print(train.shape)
print(test.shape)


In [ ]:
# Basic model using OLS
formula_basic = 'lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'
model_basic = smf.ols(formula_basic, data=train).fit()
print("Number of regressors in the basic model:", len(model_basic.params))
print(model_basic.summary())

In [ ]:
# Flexible model using OLS
formula_flex = 'lwage ~ sex + shs+hsg+scl+clg+occ2+ind2+mw+so+we + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)'
model_flex = smf.ols(formula_flex, data=train).fit()
print("Number of regressors in the flexible model:", len(model_flex.params))

In [ ]:
# Flexible model using Lasso, in-sample fit
y_train = train['lwage']
X_train = pd.get_dummies(train, drop_first=True)
lasso_cv = LassoCV(alphas=alphas, cv=5).fit(X_train, y_train)

In [ ]:
# Results storage
results = pd.DataFrame({
    'alpha': lasso_cv.alphas_,
    'mse': np.mean(lasso_cv.mse_path_, axis=1),
    'lambda_min': lasso_cv.alpha_,
    'nzero': np.sum(lasso_cv.coef_ != 0)
})

In [ ]:
# Display results
results_sorted = results.sort_values(by='mse')
print(results_sorted)

In [ ]:
# Saving optimal alpha
alpha_opt=  results_sorted.iloc[0]['alpha']
alpha_opt

In [ ]:
# Plotting the results
sns.lineplot(data=results_sorted, x='alpha', y='mse')
plt.title('Cross-validated MSE for each Alpha')
plt.show()

### Data Splitting: Out-of-sample performance

Now that we have seen in-sample fit, we evaluate our models on the out-of-sample performance:

1. Use the testing sample for evaluation. Predict the **wage** of every observation in the testing sample based on the estimated parameters in the training sample.
2. Calculate the Mean Squared Prediction Error (**MSE<sub>test</sub>**) based on the testing sample for both prediction models.



In [ ]:
# Out-of-sample performance
# calculating the out-of-sample MSE
test = sm.add_constant(test)   #add constant 

lwage_pred =  model_basic.predict(test) # predict out of sample
#print(lwage_pred)

In [ ]:
MSE_test1 = np.sum((lwage_test-lwage_pred)**2)/len(lwage_test)
R2_test1  = 1 - MSE_test1/np.var(lwage_test)

print("Test MSE for the basic model: ", MSE_test1, " ")
print("Test R2 for the basic model: ", R2_test1)

In [ ]:
# Flexible model
# estimating the parameters in the training sample
flex_results = smf.ols(formula_flex  , data=train).fit()

# calculating the out-of-sample MSE
lwage_flex_pred =  flex_results.predict(test) # predict out of sample
lwage_test = test["lwage"].values

MSE_test2 = np.sum((lwage_test-lwage_flex_pred)**2)/len(lwage_test)
R2_test2  = 1 - MSE_test2/np.var(lwage_test)

print("Test MSE for the flexible model: ", MSE_test2, " ")
print("Test R2 for the flexible model: ", R2_test2)

In [ ]:
# flexible model using lasso
# get exogenous variables from training data used in flex model
flex_results_0 = smf.ols(formula_flex , data=train)
X_train = flex_results_0.exog
print(X_train.shape)

# Get endogenous variable 
lwage_train = train["lwage"]
print(lwage_train.shape)


In [ ]:
# Flexible model using Lasso
# get exogenous variables from testing data used in flex model
flex_results_1 = smf.ols(formula_flex , data=test)
X_test = flex_results_1.exog
print(X_test.shape)

# Get endogenous variable 
lwage_test = test["lwage"]
print(lwage_test.shape)

In [ ]:
# calculating the out-of-sample MSE
reg = linear_model.Lasso(alpha=alpha_opt)
lwage_lasso_fitted = reg.fit(X_train, lwage_train).predict( X_test )

MSE_lasso = np.sum((lwage_test-lwage_lasso_fitted)**2)/len(lwage_test)
R2_lasso  = 1 - MSE_lasso/np.var(lwage_test)

print("Test MSE for the flexible model: ", MSE_lasso, " ")
print("Test R2 for the flexible model: ", R2_lasso)

In [ ]:
# Print MSE and R^2

table2 = np.zeros((3, 2))
table2[0,0] = MSE_test1
table2[1,0] = MSE_test2
table2[2,0] = MSE_lasso
table2[0,1] = R2_test1
table2[1,1] = R2_test2
table2[2,1] = R2_lasso

table2 = pd.DataFrame(table2, columns = ["$MSE_{test}$", "$R^2_{test}$"], \
                      index = ["basic reg","flexible reg","lasso regression"])
table2

# **1.Frisch-Waugh-Lovell (FWL) Theorem Proof**

Given a linear regression model, we aim to demonstrate the FWL theorem using the following elements:

- **$y$**: dependent variable vector ($n \times 1$)
- **$D$**: matrix of independent variables of interest ($n \times k_1$)
- **$\beta_1$**: coefficient vector for $D$ ($k_1 \times 1$)
- **$W$**: matrix of control variables ($n \times k_2$)
- **$\beta_2$**: coefficient vector for $W$ ($k_2 \times 1$)
- **$u$**: error term vector ($n \times 1$)

The model is represented as:

$$ y = D\beta_1 + W\beta_2 + u$$

---

## **Objective**

To prove that $\Psi = \beta_1$ can be accurately estimated through the regression $e_y = e_D \Psi + \varepsilon$, employing the FWL theorem.

---

## **Proof**

### **Step 1: Control for Variables in $W$**

First, we calculate the residuals after controlling for $W$:

- **Regress $D$ on $W$:** Aim to determine the component of $D$ that is orthogonal to $W$. This is achieved by calculating the residuals $e_D$, using the projection matrix:
  
  $$M_W = I - W(W'W)^{-1}W'$$
  
  Thus, the residuals for $D$ are:
  
  $$e_D = M_W D$$ 

- **Regress $y$ on $W$:** Similarly, find the component of $y$ not explained by $W$:
  
  $$e_y = M_W y$$

### **Step 2: Estimate $\Psi$**

With the residuals obtained, we proceed to estimate $\Psi$:

- **Regress $e_y$ on $e_D$ by OLS:** 

  $$ e_y = e_D \Psi + z $$
  
  Solving for $\Psi$, we get:
  
  $$ \hat{\Psi} = (e_D'e_D)^{-1}e_D'e_y $$
  
  Substituting the expressions for residuals into $\hat{\Psi}$ yields:
  
  $$ \hat{\Psi} = (D'M_W'M_W'D)^{-1}D'M_W'M_Wy $$
  
  Which simplifies to:
  
  $$ \hat{\Psi} = (D'M_WD)^{-1}D'M_Wy $$

And this proof that $\Psi = \beta_1$.







# **2.Conditional Expectation Function Minimizes Expected Squared Error Proof**

## **Problem Statement**

Given a random variable $Y$ and a conditioning variable $X$, we consider a relationship of the form:
$$ Y = m(X) + \epsilon $$
where:
- $m(X) = E[Y | X]$ is the Conditional Expectation Function (CEF) of $Y$ given $X$.
- $\epsilon$ is the error term, representing the deviation of $Y$ from its conditional mean.

## **Objective**

Our goal is to prove that the function that minimizes the expected squared error:
$$ m(X) = \text{arg}\min_{g(X)} E[(Y - g(X))^2] $$

is indeed:
$$ E[(Y - g(X))^2] = E[\epsilon^2] $$

## **Proof**

### **Step 1: Expanding the Expected Squared Error**

We start by expanding the expected squared difference as follows:
$$ E[(Y - g(X))^2] = E[(Y - E[Y|X] + E[Y|X] - g(X))^2] $$

By applying the expansion for the square of a sum $(a + b)^2 = a^2 + b^2 + 2ab$, where $a = Y - E[Y|X]$ and $b = E[Y|X] - g(X)$, we obtain:
$$ E[(Y - g(X))^2] = E[(Y - E[Y|X])^2] + E[(E[Y|X] - g(X))^2] + 2E[(Y - E[Y|X])(E[Y|X] - g(X))] $$

### **Step 2: Simplifying Using the Law of Iterated Expectations**

Applying the Law of Iterated Expectations to the mixed term:
$$ 2E[(Y - E[Y|X])(E[Y|X] - g(X))] = 0 $$

This follows because the expectation of $Y - E[Y|X]$ is zero by definition of the error term $\epsilon$ (i.e., $Y - E[Y|X] = \epsilon$ and $E[\epsilon] = 0$), and thus the cross-term disappears.

### **Step 3: Final Reduction**

After removing the cross-term, we are left with:
$$ E[(Y - g(X))^2] = E[(Y - E[Y|X])^2] + E[(E[Y|X] - g(X))^2] $$

Since the second term $E[(E[Y|X] - g(X))^2]$ is always non-negative, it follows that:
$$ E[(Y - g(X))^2] \geq E[(Y - E[Y|X])^2] $$

### **Conclusion**

The expected squared error is minimized when $g(X) = E[Y|X]$, demonstrating that the Conditional Expectation Function (CEF) $m(X)$ minimizes the expected squared error:
$$ m(X) = \text{arg}\min_{g(X)} E[(Y - g(X))^2] $$
This conclusively proves that the CEF is the function that minimizes the expected squared error between the predicted values and the actual values of $Y$.
